In [0]:
from pyspark.sql.functions import expr, sequence, explode, col, date_format, year, month, dayofmonth, current_timestamp

In [0]:
catalog_name = "webinar"
start_date = "2023-01-01"
end_date   = "2025-12-31"

df = (
    spark.range(1)
    .withColumn("fecha", explode(sequence(expr(f"to_date('{start_date}')"),
                                            expr(f"to_date('{end_date}')"),
                                            expr("interval 1 day"))))
    .select("fecha")
    .withColumn("anio", year(col("fecha")))
    .withColumn("mes", month(col("fecha")))
    .withColumn("dia", dayofmonth(col("fecha")))
    .withColumn("anio_mes", date_format(col("fecha"), "yyyyMM"))
    .withColumn("sk_tiempo", date_format(col("fecha"), "yyyyMMdd").cast("int"))
    .withColumn("updated_at", current_timestamp())
    .select("sk_tiempo", "fecha", "anio", "mes", "dia", "anio_mes", "updated_at")
)

df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog_name}.gold.tiempo")